# Chapter 0: Introduction to Engineering Software for Mechatronic Systems

---
### Python as a Language
- High-level, interpreted, dynamically typed
- Supports multiple paradigms: Procedural, Object-oriented, and Functional
- **Batteries included:** rich standard library
- Industry standard for robotics (ROS, simulation, data science)
- Readable syntax that encourages clean code

## 1. Encapsulation & Abstraction

Encapsulation bundles the data (reading) and methods (calibrate) together, while Abstraction hides the complex math inside a simple method call.

In [1]:
class Sensor:
    def __init__(self, name):
        self.name = name
        self.__offset = 0.5  # Encapsulation: Private variable (hidden)

    def get_calibrated_value(self, raw_value):
        # Abstraction: User doesn't need to know the formula
        return raw_value - self.__offset

temp_sensor = Sensor("TMP36")
print(temp_sensor.get_calibrated_value(25.8))

25.3


## 2. Inheritance

Inheritance allows a specific type of sensor to "inherit" the properties of a general sensor without rewriting code.

In [2]:
class UltrasonicSensor(Sensor):
    def __init__(self, name, pin):
        super().__init__(name) # Inherit from parent
        self.pin = pin

    def ping(self):
        return "Sending sonic pulse..."

dist_sensor = UltrasonicSensor("HC-SR04", 12)
print(dist_sensor.name) # Accessed from parent
print(dist_sensor.ping()) # Own unique method

HC-SR04
Sending sonic pulse...


## 3. Polymorphism

Polymorphism allows different objects to be treated the same way. In mechatronics, you might want to "read" all sensors in a loop, regardless of whether they are Temperature, Pressure, or Light sensors.

In [4]:
class TemperatureSensor:
    def read(self):
        return "24°C"

class PressureSensor:
    def read(self):
        return "101.3 kPa"

# Polymorphism in action
sensors = [TemperatureSensor(), PressureSensor()]

for s in sensors:
    print(s.read()) # Same method name, different behaviors

24°C
101.3 kPa


## Think of it like a CNC machine: 
You give it a high-level command like move_to(x, y), and you don't need to know the specific electrical pulses sent to the stepper motors, the current limits, or the internal PID loop logic. You only care about the result.
The Two Layers of Abstraction
- The Interface (Public): What the user sees and interacts with (the "What").
- The Implementation (Private): The complex logic hidden "under the hood" (the "How").

## Python Example: A Motor Controller

In this example, we use an Abstract Base Class (ABC). This acts as a blueprint or "contract" that ensures any type of motor we build later will have the same basic commands.

In [7]:
from abc import ABC, abstractmethod

# The Abstract Class (The Blueprint)
class Motor(ABC):
    @abstractmethod
    def set_speed(self, rpm):
        """Every motor MUST implement this method"""
        pass

    @abstractmethod
    def stop(self):
        """Every motor MUST implement this method"""
        pass

# The Concrete Implementation (The Hidden Complexity)
class StepperMotor(Motor):
    def set_speed(self, rpm):
        # Implementation details hidden from the user:
        # 1. Calculate pulse frequency
        # 2. Check current limits
        # 3. Write to GPIO pins
        print(f"Stepper Motor: Calculating steps for {rpm} RPM...")
        
    def stop(self):
        print("Stepper Motor: Cutting power to coils.")

# Using the Abstraction
# The user doesn't care HOW the motor spins, only that it HAS a set_speed method.
my_motor = StepperMotor()
my_motor.set_speed(1500)
my_motor.stop()

Stepper Motor: Calculating steps for 1500 RPM...
Stepper Motor: Cutting power to coils.


## Fully Abstraction Class

In [8]:
from abc import ABC, abstractmethod

# 1. THE ABSTRACTION (The Blueprint)
class Motor(ABC):
    @abstractmethod
    def set_speed(self, rpm):
        """Every motor MUST implement this method"""
        pass

    @abstractmethod
    def stop(self):
        """Every motor MUST implement this method"""
        pass

# 2. CONCRETE IMPLEMENTATION A: Stepper Motor
class StepperMotor(Motor):
    def set_speed(self, rpm):
        # Hidden complexity: Step logic
        print(f">>> [Hardware Call] Pulsing GPIO pins for {rpm} RPM.")
        print(f"Stepper Motor: Moving at {rpm} RPM.")
        
    def stop(self):
        print("Stepper Motor: Engaging holding torque and stopping.")

# 3. CONCRETE IMPLEMENTATION B: DC Motor
class DCMotor(Motor):
    def set_speed(self, rpm):
        # Hidden complexity: PWM (Pulse Width Modulation) logic
        voltage = rpm * 0.01 
        print(f">>> [Hardware Call] Setting PWM Duty Cycle to {voltage}V.")
        print(f"DC Motor: Spinning at {rpm} RPM.")
        
    def stop(self):
        print("DC Motor: Shorting terminals for dynamic braking.")

# --- EXECUTION (The Input) ---

print("--- Testing Stepper ---")
my_stepper = StepperMotor()
my_stepper.set_speed(1500)
my_stepper.stop()

print("\n--- Testing DC Motor ---")
my_dc = DCMotor()
my_dc.set_speed(3000)
my_dc.stop()

--- Testing Stepper ---
>>> [Hardware Call] Pulsing GPIO pins for 1500 RPM.
Stepper Motor: Moving at 1500 RPM.
Stepper Motor: Engaging holding torque and stopping.

--- Testing DC Motor ---
>>> [Hardware Call] Setting PWM Duty Cycle to 30.0V.
DC Motor: Spinning at 3000 RPM.
DC Motor: Shorting terminals for dynamic braking.
